In [33]:
#!pip install -U transformers
#!pip install -U datasets
import os
import sys
HOME = os.path.abspath('..')
sys.path.append(HOME)
os.chdir(HOME)
import pandas as pd
#!pip install transformers
from transformers import RobertaConfig, RobertaModel,RobertaForSequenceClassification, Trainer,AutoModelForSequenceClassification, EarlyStoppingCallback 
from transformers import AutoTokenizer
from transformers.models.roberta import RobertaPreTrainedModel
import torch
from torch import nn
from transformers import TrainingArguments
import shutil
import glob

In [34]:
MODEL_NAME = "distilbert-base-uncased"
TARGET_COL = 'averageRating'
COLAB = True

if COLAB == True:
  if not os.path.exists('data'):
    os.mkdir('data')
  if not os.path.exists('data/processed'):
    os.mkdir('data/processed')

  drive.mount('/content/gdrive/')
  for filename in glob.glob(os.path.join('gdrive/MyDrive/atdl', '*.*')):
      shutil.copy(filename, 'data/processed')

NameError: name 'drive' is not defined

In [35]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

class IMDbDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def process_text_data(data_:pd.DataFrame,text_col,padding ="max_length", truncation = True, na_filler = ""):

    '''
    
    
    '''
    data = data_.copy()
    data[text_col] = data[text_col].fillna(na_filler)
    encodings = tokenizer(data[text_col].tolist(), padding=padding, truncation=truncation)
    return encodings
    

def columns_to_single_text(df,cols_to_transform,new_col_name = 'text_input',sep = tokenizer.sep_token,nan_replacement = tokenizer.unk_token ):

  '''
  
  Creates a new column called new_col_name with with all columns in cols_to_transform concatenated into a single text
  '''
  df[new_col_name] = df[cols_to_transform].astype(str).replace('nan',nan_replacement).agg(f' {sep} '.join, axis=1)


class NAFiller:

  def __init__(self,train):
    self.train = train

  def fit(self,column = 'Budget',groupby=['top_genre','top_country']):
    self.mapping = self.train.groupby(groupby)[column].median().reset_index()
    self.mapping = self.mapping.rename(columns={column:'na_filler'})
    self.median = self.train[column].median()
    self.column=column


  def transform(self,test,round = False):
    self.na_filler = test.merge(self.mapping,how='left')['na_filler']
    self.na_filler = self.na_filler.fillna(self.median)

    test[self.column] = test[self.column].reset_index(drop=True).fillna(self.na_filler).values

    if round:
      test[self.column].round().astype(int)
      


  def fit_transform(self,test,column = 'Budget',groupby=['top_genre','top_country']):
    self.fit(column,groupby)
    self.transform()
    

def create_dataset_split(split,text_cols,text_input_col,TARGET_COL):

  #If all columns in text_cols are combined into a single text. A n
  columns_to_single_text(split,text_cols)

  #Get split encodings
  split_encodings = process_text_data(split,text_input_col)

  #get labels
  split_labels = split[TARGET_COL].tolist()

  #Create dataset objects
  split_dataset = IMDbDataset(split_encodings, split_labels)

  return split_dataset

In [36]:
all_cols =  ['Budget',
             'averageRating',
             'cast',
             'countries',
             'director',
             'genres',
             'imdb_id',
             'languages',
             'overview',
             'production companies',
             'release_date',
             'revenue_worldwide_BOM',
             'runtimeMinutes',
             'title']

categoric_cols = ['cast',
                  'countries',
                  'director',
                  'genres',
                  'languages',
                  'production companies']

text_cols = ['title','overview']                  
date_cols = ['release_date']
numeric_cols = ['Budget','runtimeMinutes']

CATEGORIES_AS_TEXT = True
NUMERIC_AS_TEXT = True
DATE_AS_TEXT = True
text_input_col = 'text_input'

if CATEGORIES_AS_TEXT:
  text_cols+=categoric_cols

if NUMERIC_AS_TEXT:
  text_cols+=numeric_cols

if DATE_AS_TEXT:
  text_cols+=date_cols


train_ids = pd.read_csv('data/processed/train.csv',usecols=['imdb_id'])['imdb_id'].tolist()
val_ids = pd.read_csv('data/processed/val.csv',usecols=['imdb_id'])['imdb_id'].tolist()
test_ids = pd.read_csv('data/processed/test.csv',usecols=['imdb_id'])['imdb_id'].tolist()
df = pd.read_csv('data/processed/df.csv',usecols = all_cols,parse_dates=['release_date']).sample(frac=1) #shuffle


df[categoric_cols] = df[categoric_cols].apply(lambda x: x.str.replace('|',', '),axis=0) #Change pipe to comma, its more meaningful

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:48: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [37]:
#Additional auxilary columns
df['top_genre'] = df['genres'].apply(lambda x: x.split(', ')[0])
df['top_country'] = df['countries'].apply(lambda x: x.split(', ')[0] if isinstance(x,str) else x)
df['year'] = df['release_date'].dt.year

In [38]:
#Create splits
train = df[df['imdb_id'].isin(train_ids)]
val = df[df['imdb_id'].isin(val_ids)]
test = df[df['imdb_id'].isin(test_ids)]


#Fill na in some columns with statistics
naf = NAFiller(train)
naf.fit(column = 'Budget',groupby=['top_genre','top_country'])
naf.transform(train,round=True)
naf.transform(val,round=True)
naf.transform(test,round=True)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
'''
#If all columns in text_cols are combined into a single text. A n
columns_to_single_text(train,text_cols)
columns_to_single_text(val,text_cols)
columns_to_single_text(test,text_cols)


#Get train encodings
train_encodings = process_text_data(train,text_input_col)
val_encodings = process_text_data(val,text_input_col)
test_encodings = process_text_data(test,text_input_col)

#get labels
train_labels = train[TARGET_COL].tolist()
val_labels = val[TARGET_COL].tolist()
test_labels = test[TARGET_COL].tolist()

#Create dataset objects
train_dataset = IMDbDataset(train_encodings, train_labels)
val_dataset = IMDbDataset(val_encodings, val_labels)
test_dataset = IMDbDataset(test_encodings, test_labels)
'''


train_dataset=create_dataset_split(train,text_cols,text_input_col,TARGET_COL)
val_dataset=create_dataset_split(val,text_cols,text_input_col,TARGET_COL)
test_dataset=create_dataset_split(test,text_cols,text_input_col,TARGET_COL)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
#device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [12]:
def get_model():
  return AutoModelForSequenceClassification.from_pretrained(MODEL_NAME,
                                                         problem_type='regression',
                                                         num_labels=1
                                                        )

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "regression",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.17.0",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert

In [13]:

batch_size = 16
gradient_accumulation_steps = 1
epochs = 6
num_evals = 20
eval_steps = (len(train)*epochs//batch_size)//num_evals
learning_rate = 5e-05
weight_decay = 0.1
callbacks=[EarlyStoppingCallback(early_stopping_patience=20)]

eval_steps = 100

training_args = TrainingArguments(output_dir=f"{MODEL_NAME}-{TARGET_COL}",
                                  #fp16=True,
                                  per_device_train_batch_size = batch_size,
                                  gradient_accumulation_steps = gradient_accumulation_steps,
                                  per_device_eval_batch_size = batch_size,
                                  learning_rate = learning_rate,
                                  weight_decay = weight_decay,
                                  num_train_epochs = epochs,
                                  evaluation_strategy = "steps",
                                  save_strategy = "steps",
                                  load_best_model_at_end=True,
                                  eval_steps = eval_steps,
                                  save_steps = eval_steps)

trainer = Trainer(
    model_init=get_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    #compute_metrics=compute_metrics,
    callbacks = callbacks
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 1

In [ ]:
trainer.train()

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "regression",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.17.0",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert

Step,Training Loss,Validation Loss
100,No log,1.442023
200,No log,1.399222
300,No log,1.089085
400,No log,0.956043
500,1.694500,1.066266
600,1.694500,1.283469
700,1.694500,0.830654
800,1.694500,0.844380
900,1.694500,0.861464
1000,0.704100,0.887585


***** Running Evaluation *****
  Num examples = 1754
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-averageRating/checkpoint-100
Configuration saved in distilbert-base-uncased-averageRating/checkpoint-100/config.json
Model weights saved in distilbert-base-uncased-averageRating/checkpoint-100/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1754
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-averageRating/checkpoint-200
Configuration saved in distilbert-base-uncased-averageRating/checkpoint-200/config.json
Model weights saved in distilbert-base-uncased-averageRating/checkpoint-200/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1754
  Batch size = 16
Saving model checkpoint to distilbert-base-uncased-averageRating/checkpoint-300
Configuration saved in distilbert-base-uncased-averageRating/checkpoint-300/config.json
Model weights saved in distilbert-base-uncased-averageRating/checkpoint-300/pytorch_model.bin
*

In [ ]:
trainer.evaluate()

In [ ]:
import numpy as np
from datasets import load_metric

metric = load_metric("mse")

def compute_metrics(eval_pred,metric):
    logits, labels = eval_pred
    predictions = logits
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
preds = trainer.predict(test_dataset)

In [ ]:
eval_preds = [preds.predictions,preds.label_ids]
compute_metrics(eval_preds,metric)

In [ ]:
y_bar = preds.label_ids.mean()
eval_preds = [preds.predictions,preds.label_ids]

1-compute_metrics(eval_preds,metric)['mse']/((preds.label_ids-y_bar)**2).mean()

In [ ]:
preds.predictions.max(),preds.predictions.min()

In [ ]:
sum((preds.label_ids>=4.17)&(preds.label_ids<=7.92))/len(preds.label_ids)

In [ ]:

'''
from itertools import product

hparams = {'learning_rate' : [1e-5, 2e-5, 3e-5],
           'weight_decay' : [0.1,0.01],
           'repeats': range(1)}

combs = list(product(*[range(len(i)) for i in list(hparams.values())]))
scores = np.zeros([len(i) for i in list(hparams.values())])


for comb_indexes in combs:
    comb_values = {name:val[idx] for name,val,idx in zip(hparams.keys(),hparams.values(),comb_indexes)}
    score = 1
    scores[tuple(comb_indexes)] = score

   ''' 

In [9]:
from itertools import product
import numpy as np

epochs = 10
num_evals = 20
patience = 2 
eval_steps = 50 

hparams = {'batch_size' : [8,16,32],
           'learning_rate' : [1e-5, 2e-5, 3e-5,5e-5],
           'weight_decay' : [0.1,0.01],
           'repeats': range(1)}

combs = list(product(*[range(len(i)) for i in list(hparams.values())]))
scores = np.zeros([len(i) for i in list(hparams.values())])
best_score = float('inf')

for idx,comb_indexes in enumerate(combs):
    comb_values = {name:val[idx] for name,val,idx in zip(hparams.keys(),hparams.values(),comb_indexes)}
    print(comb_values)

{'batch_size': 8, 'learning_rate': 1e-05, 'weight_decay': 0.1, 'repeats': 0}
{'batch_size': 8, 'learning_rate': 1e-05, 'weight_decay': 0.01, 'repeats': 0}
{'batch_size': 8, 'learning_rate': 2e-05, 'weight_decay': 0.1, 'repeats': 0}
{'batch_size': 8, 'learning_rate': 2e-05, 'weight_decay': 0.01, 'repeats': 0}
{'batch_size': 8, 'learning_rate': 3e-05, 'weight_decay': 0.1, 'repeats': 0}
{'batch_size': 8, 'learning_rate': 3e-05, 'weight_decay': 0.01, 'repeats': 0}
{'batch_size': 8, 'learning_rate': 5e-05, 'weight_decay': 0.1, 'repeats': 0}
{'batch_size': 8, 'learning_rate': 5e-05, 'weight_decay': 0.01, 'repeats': 0}
{'batch_size': 16, 'learning_rate': 1e-05, 'weight_decay': 0.1, 'repeats': 0}
{'batch_size': 16, 'learning_rate': 1e-05, 'weight_decay': 0.01, 'repeats': 0}
{'batch_size': 16, 'learning_rate': 2e-05, 'weight_decay': 0.1, 'repeats': 0}
{'batch_size': 16, 'learning_rate': 2e-05, 'weight_decay': 0.01, 'repeats': 0}
{'batch_size': 16, 'learning_rate': 3e-05, 'weight_decay': 0.1, 'r

In [16]:
scores_avg = scores.mean(axis=-1)

In [22]:
import pandas as pd

In [23]:
pd.DataFrame([comb_values,comb_values])

,batch_size,learning_rate,weight_decay,repeats
0,32,0.00005,0.01,0
1,32,0.00005,0.01,0


In [29]:
rows=[]
for idx,comb_indexes in enumerate(combs):
    row = {name:val[idx] for name,val,idx in zip(hparams.keys(),hparams.values(),comb_indexes)}
    row['score']=scores[tuple(comb_indexes)]
    rows.append(row)
trials_df = pd.DataFrame(rows)

In [30]:
trials_df

,batch_size,learning_rate,weight_decay,repeats,score
0,8,0.00001,0.10,0,0.0
1,8,0.00001,0.01,0,0.0
2,8,0.00002,0.10,0,0.0
3,8,0.00002,0.01,0,0.0
4,8,0.00003,0.10,0,0.0
5,8,0.00003,0.01,0,0.0
6,8,0.00005,0.10,0,0.0
7,8,0.00005,0.01,0,0.0
8,16,0.00001,0.10,0,0.0
9,16,0.00001,0.01,0,0.0


In [28]:
row

{'batch_size': 32,
 'learning_rate': 5e-05,
 'weight_decay': 0.01,
 'repeats': 0,
 'score': 0.0}

In [44]:
((val['averageRating']-val['averageRating'].mean())**2).mean()

1.2757810718358051

In [50]:
1-0.5/1.27

0.6062992125984252

In [51]:
((test['averageRating']-test['averageRating'].mean())**2).mean()

1.1645053430568955

In [54]:
1-0.4/1.16

0.6551724137931034